In [1]:
# Import necessary dependencies
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Check the version of TensorFlow
print(tf.__version__)

1.6.0


In [3]:
x = np.arange(0, 10)
print(x)

[0 1 2 3 4 5 6 7 8 9]


In [4]:
# Create dataset object from numpy array
dx = tf.data.Dataset.from_tensor_slices(x)
print(dx)

<TensorSliceDataset shapes: (), types: tf.int64>


In [5]:
# Create a one-shot iterator which can only be initialized and run once
iterator = dx.make_one_shot_iterator()

In [6]:
# Extract an element
next_element = iterator.get_next()
print(next_element)

Tensor("IteratorGetNext:0", shape=(), dtype=int64)


In [7]:
with tf.Session() as sess:
  for i in range(10):
    val = sess.run(next_element)
    print(val)

0
1
2
3
4
5
6
7
8
9


In [11]:
# If we want to repeatedly extract data from a dataset
iterator = dx.make_initializable_iterator()
next_element = iterator.get_next()

In [12]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(15):
        val = sess.run(next_element)
        print(val)
        if i % 9 == 0 and i > 0:
            sess.run(iterator.initializer)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [15]:
# Using batches
dx = tf.data.Dataset.from_tensor_slices(x).batch(3)
iterator = dx.make_initializable_iterator()
next_element = iterator.get_next()

In [16]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(15):
        val = sess.run(next_element)
        print(val)
        if (i + 1) % (10 // 3) == 0 and i > 0:
            sess.run(iterator.initializer)

[0 1 2]
[3 4 5]
[6 7 8]
[0 1 2]
[3 4 5]
[6 7 8]
[0 1 2]
[3 4 5]
[6 7 8]
[0 1 2]
[3 4 5]
[6 7 8]
[0 1 2]
[3 4 5]
[6 7 8]


In [17]:
"""
Zip together datasets. 
This is useful when pairing up input-output training/validation pairs of data (i.e. input images and matching labels for each image). 
"""
def simple_zip_example():
    x = np.arange(0, 10)
    y = np.arange(1, 11)
    # Create dataset objects from the arrays
    dx = tf.data.Dataset.from_tensor_slices(x)
    dy = tf.data.Dataset.from_tensor_slices(y)
    # Zip the two datasets together
    dcomb = tf.data.Dataset.zip((dx, dy)).batch(3)
    iterator = dcomb.make_initializable_iterator()
    # Extract an element
    next_element = iterator.get_next()
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        for i in range(15):
            val = sess.run(next_element)
            print(val)
            if (i + 1) % (10 // 3) == 0 and i > 0:
                sess.run(iterator.initializer)

In [18]:
simple_zip_example()

(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))


In [19]:
# Or we can also do it repeatly
def simple_zip_example2():
    x = np.arange(0, 10)
    y = np.arange(1, 11)
    # Create dataset objects from the arrays
    dx = tf.data.Dataset.from_tensor_slices(x)
    dy = tf.data.Dataset.from_tensor_slices(y)
    # Zip the two datasets together
    dcomb = tf.data.Dataset.zip((dx, dy)).repeat().batch(3)
    iterator = dcomb.make_initializable_iterator()
    # Extract an element
    next_element = iterator.get_next()
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        for i in range(15):
            val = sess.run(next_element)
            print(val)

In [20]:
simple_zip_example2()

(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([9, 0, 1]), array([10,  1,  2]))
(array([2, 3, 4]), array([3, 4, 5]))
(array([5, 6, 7]), array([6, 7, 8]))
(array([8, 9, 0]), array([ 9, 10,  1]))
(array([1, 2, 3]), array([2, 3, 4]))
(array([4, 5, 6]), array([5, 6, 7]))
(array([7, 8, 9]), array([ 8,  9, 10]))
(array([0, 1, 2]), array([1, 2, 3]))
(array([3, 4, 5]), array([4, 5, 6]))
(array([6, 7, 8]), array([7, 8, 9]))
(array([9, 0, 1]), array([10,  1,  2]))
(array([2, 3, 4]), array([3, 4, 5]))


In [21]:
# A simple Neural Network example
from sklearn.datasets import load_digits

In [22]:
# Load the data
digits = load_digits(return_X_y=True)

In [23]:
# Split into train and validation sets
train_images = digits[0][:int(len(digits[0]) * 0.8)]
train_labels = digits[1][:int(len(digits[0]) * 0.8)]
valid_images = digits[0][int(len(digits[0]) * 0.8):]
valid_labels = digits[1][int(len(digits[0]) * 0.8):]

In [25]:
print(train_images.shape)
print(train_labels.shape)

(1437, 64)
(1437,)


In [26]:
# Create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(train_images)
# Apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(train_labels).map(lambda z: tf.one_hot(z, 10))
# Zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(30)

In [27]:
# Do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(valid_images)
dy_valid = tf.data.Dataset.from_tensor_slices(valid_labels).map(lambda z: tf.one_hot(z, 10))
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

In [29]:
# Create general iterator
print(train_dataset.output_types)
print(train_dataset.output_shapes)
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
next_element = iterator.get_next()

(tf.float64, tf.float32)
(TensorShape([Dimension(None), Dimension(64)]), TensorShape([Dimension(None), Dimension(10)]))


In [30]:
# Make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [33]:
# Define the simple Neural Network Model
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 50)
    fc2 = tf.layers.dense(fc1, 50)
    fc2 = tf.layers.dropout(fc2)
    fc3 = tf.layers.dense(fc2, 10)
    return fc3

In [34]:
"""
The next_element operation, because it is operating on the generic iterator which is defined by the shape of the train_dataset, 
is a tuple – the first element ([0]) will contain the MNIST images, while the second element ([1]) will contain the corresponding labels.
"""
logits = nn_model(next_element[0])

In [35]:
# Add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [36]:
# Get accuracy
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
init_op = tf.global_variables_initializer()

In [39]:
# Run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print(f"Epoch: {i}, loss: {l}, training accuracy: {acc}")
    # Now set up the validation run
    valid_iters = 100
    # Re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0] / valid_iters
    print(f"Average validation set accuracy over {i} iterations is {avg_acc}")

Epoch: 0, loss: 368.2855408171384, training accuracy: 0.10000000149011612
Epoch: 50, loss: 8.232498104386487, training accuracy: 0.9333333373069763
Epoch: 100, loss: 5.240766030038692, training accuracy: 0.9666666388511658
Epoch: 150, loss: 4.337822829377826, training accuracy: 0.9333333373069763
Epoch: 200, loss: 1.2398580430939572, training accuracy: 1.0
Epoch: 250, loss: 3.290698012202787, training accuracy: 0.9666666388511658
Epoch: 300, loss: 1.3161388262902953, training accuracy: 1.0
Epoch: 350, loss: 2.0153298106564628, training accuracy: 0.9666666388511658
Epoch: 400, loss: 3.1704537747950354, training accuracy: 0.9666666388511658
Epoch: 450, loss: 2.732271721085435, training accuracy: 0.9666666388511658
Epoch: 500, loss: 1.865572943875488, training accuracy: 0.9666666388511658
Epoch: 550, loss: 4.0344662773575415, training accuracy: 0.9333333373069763
Average validation set accuracy over 99 iterations is 0.8966666567325586
